<a href="https://colab.research.google.com/github/AG0nzales/Language_Identifier/blob/main/Language_Identifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cebuano Language Identifier

## Imports

In [ ]:
import numpy as np
import pandas as pd
import string
import re
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

# Reading Dataframe

In [ ]:
df = pd.read_excel('/content/Parallel.OpenDomain.xlsx', names=["Contexts", "Language"])
df

,Contexts,Language
0,Ang puso niya'y puno nang kaligayahan.,Filipino
1,Ilan ang mga kulay?,Filipino
2,Nakakapagbasa ako ng Ingles.,Filipino
3,Nakikita ko sa mata mo.,Filipino
4,Di ko rin maipaliwanag.,Filipino
...,...,...
19994,Ang Dakila nga Pula nga Batik sa Hupiter kay d...,Cebuano
19995,Ang Saturno kay naay ruweda nga nagalibot sa i...,Cebuano
19996,Hapit mag 150 ka bulan ang nagalibot sa Saturno.,Cebuano
19997,Gihigugma nako akoang amahan.,Cebuano


# Data Pre-processing

In [ ]:
def remove_pun(text):
  for pun in string.punctuation:
    text = str(text).replace(pun, "")
  text = text.lower()
  return text

In [ ]:
df['Contexts'] = df['Contexts'].apply(remove_pun)

In [ ]:
Filip = df[df['Language'] == 'Filipino'] 

Cebua = df[df['Language'] == 'Cebuano'] 

In [ ]:
Filip['tokenized_conts'] = Filip.apply(lambda row: nltk.word_tokenize(row['Contexts']), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
Cebua['tokenized_contz'] = Cebua.apply(lambda row: nltk.word_tokenize(row['Contexts']), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
Filip['conts_length'] = Filip.apply(lambda row: len(row['tokenized_conts']), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
Cebua['contz_length'] = Cebua.apply(lambda row: len(row['tokenized_contz']), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
Filip['totalwords'] = [len(x.split()) for x in Filip['Contexts'].tolist()]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
Cebua['totalwords'] = [len(x.split()) for x in Cebua['Contexts'].tolist()]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
Total = Filip['totalwords'].sum()
print (Total)

63201


In [ ]:
Total2 = Cebua['totalwords'].sum()
print (Total2)

66339


In [ ]:
def Average(lst):
    return sum(lst) / len(lst) 

In [ ]:
W = []
uniquetoken = 0
words = 0
for text in Filip['Contexts']:
  token = text.split()
  a_set = set(token)
  number_of_unique_token = len(a_set)
  uniquetoken = number_of_unique_token + uniquetoken
  ans1= len(token)
  words = words + ans1
  W.append(ans1)

print("(Filipino) Number of all contexts: ",len(W)) 
print("(Filipino) Number of tokens: ",words)
print("(Filipino) Number of unique tokens: ",uniquetoken) 
average = Average(W)
print("(Filipino) Average tokens per line: ", round(average, 2))
print("(Filipino) Maximum number of tokens in a line: ", Filip['conts_length'].max())
print("(Filipino) Minimum number of tokens in a line: ", Filip['conts_length'].min())
gret = Filip[Filip['conts_length'] > 50]
print("(Filipino) Sentences with number of tokens greater than 50: ", gret.count()[0])

(Filipino) Number of all contexts:  9999
(Filipino) Number of tokens:  63201
(Filipino) Number of unique tokens:  61352
(Filipino) Average tokens per line:  6.32
(Filipino) Maximum number of tokens in a line:  44
(Filipino) Minimum number of tokens in a line:  1
(Filipino) Sentences with number of tokens greater than 50:  0


In [ ]:
W = []
uniquetoken = 0
words = 0
for text in Cebua['Contexts']:
  token = text.split()
  a_set = set(token)
  number_of_unique_token = len(a_set)
  uniquetoken = number_of_unique_token + uniquetoken
  ans1= len(token)
  words = words + ans1
  W.append(ans1)

print("(Cebuano) Number of all contexts =",len(W)) 
print("(Cebuano) Number of tokens =",words)
print("(Cebuano) Number of unique tokens =",uniquetoken) 
average = Average(W)
print("(Cebuano) Average tokens per line =", round(average, 2))
print("(Cebuano) Maximum number of tokens in a line: ", Cebua['contz_length'].max())
print("(Cebuano) Minimum number of tokens in a line: ", Cebua['contz_length'].min())
greit = Cebua[Cebua['contz_length'] > 50]
print("(Cebuano) Sentences with number of tokens greater than 50: ", greit.count()[0])

(Cebuano) Number of all contexts = 10000
(Cebuano) Number of tokens = 66339
(Cebuano) Number of unique tokens = 63832
(Cebuano) Average tokens per line = 6.63
(Cebuano) Maximum number of tokens in a line:  47
(Cebuano) Minimum number of tokens in a line:  1
(Cebuano) Sentences with number of tokens greater than 50:  0


# Training and Testing section

In [ ]:
from sklearn.model_selection import train_test_split

# We locate where the X and Y is in the Dataframe
## preparation for training and testing

In [ ]:
X = df.iloc[:,0]
Y = df.iloc[:,1]

## Actual training and testing

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = .2)

In [ ]:
X_test

5571        ditoy may restawran na kinakainan kong malimit
2940     alam mo ba kung saan paparoon ang masamang tao...
18011                  nagdamgo ko nga maka adto sa mexico
1040              baka may nagkamaling kumuha ng payong ko
12485                          maayo ang sushi sa tindahan
                               ...                        
9777               nakapunta ako diyan nang maraming beses
7462                  kailangan mong matutong magbisikleta
13946                 dili siya nawong sa tinuod nga kapoy
10379                dili kabalo si tom magtali sa kurbata
16886                                     gahi siya og ulo
Name: Contexts, Length: 4000, dtype: object

In [ ]:
from sklearn import feature_extraction

# Vectorize

In [ ]:
vec = feature_extraction.text.TfidfVectorizer(ngram_range=(2,3), analyzer='char')

In [ ]:
from sklearn import pipeline
from sklearn import linear_model
from sklearn import tree

## LOGISTIC REGRESSION model using pipeline

In [ ]:
# clf is the name of the pipeline component
model_pipe = pipeline.Pipeline([('vec', vec), ('clf', linear_model.LogisticRegression())]) 

In [ ]:
model_pipe.fit(X_train, Y_train)

Pipeline(steps=[('vec', TfidfVectorizer(analyzer='char', ngram_range=(2, 3))),
                ('clf', LogisticRegression())])

In [ ]:
predict_val = model_pipe.predict(X_test)

In [ ]:
from sklearn import metrics

In [ ]:
metrics.accuracy_score(Y_test,predict_val)*100

94.825

In [ ]:
metrics.confusion_matrix(Y_test, predict_val)

array([[1865,  114],
       [  93, 1928]])

In [ ]:
model_pipe.predict(['AMbot talga bhie, bakit mna siya ingun ana'])

array(['Cebuano'], dtype=object)

# SVM model using pipeline

### Support Vector Machine, A supervised machine learning algorithm used for both classification and regression

In [ ]:
model_pipe2 = pipeline.Pipeline([('vec', vec), ('clf',  svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto'))])

In [ ]:
model_pipe2.fit(X_train, Y_train)

Pipeline(steps=[('vec', TfidfVectorizer(analyzer='char', ngram_range=(2, 3))),
                ('clf', SVC(gamma='auto', kernel='linear'))])

In [ ]:
predict_val2 = model_pipe2.predict(X_test)

In [ ]:
metrics.accuracy_score(Y_test,predict_val2)*100

95.39999999999999

In [ ]:
metrics.confusion_matrix(Y_test, predict_val2)

array([[1870,  109],
       [  75, 1946]])

In [ ]:
model_pipe2.predict(['AMbot talga bhie, bakit mna siya ingun ana'])

array(['Cebuano'], dtype=object)

# KNN model using pipeline

### A supervised machine learning algorithm. The algorithm can be used to solve both classification and regression problem statements

In [ ]:
model_pipe3 = pipeline.Pipeline([('vec', vec), ('clf',  KNeighborsClassifier(n_neighbors=3))])

In [ ]:
model_pipe3.fit(X_train, Y_train)

Pipeline(steps=[('vec', TfidfVectorizer(analyzer='char', ngram_range=(2, 3))),
                ('clf', KNeighborsClassifier(n_neighbors=3))])

In [ ]:
predict_val3 = model_pipe3.predict(X_test)

In [ ]:
metrics.accuracy_score(Y_test,predict_val3)*100

84.025

In [ ]:
metrics.confusion_matrix(Y_test, predict_val3)

array([[1609,  370],
       [ 269, 1752]])

In [ ]:
model_pipe3.predict(['AMbot talga bhie, bakit mna siya ingun ana'])

array(['Cebuano'], dtype=object)

# DECISION TREE model using pipeline

### It breaks down a dataset into smaller and smaller subsets while at the same time an associated decision tree is incrementally developed

In [ ]:
model_pipe4 = pipeline.Pipeline([('vec', vec), ('clf',  DecisionTreeClassifier(random_state=0))])

In [ ]:
model_pipe4.fit(X_train, Y_train)

Pipeline(steps=[('vec', TfidfVectorizer(analyzer='char', ngram_range=(2, 3))),
                ('clf', DecisionTreeClassifier(random_state=0))])

In [ ]:
predict_val4 = model_pipe4.predict(X_test)

In [ ]:
metrics.accuracy_score(Y_test,predict_val4)*100

86.325

In [ ]:
metrics.confusion_matrix(Y_test, predict_val4)

array([[1699,  280],
       [ 267, 1754]])

In [ ]:
model_pipe4.predict(['AMbot talga bhie, bakit mna siya ingun ana'])

array(['Cebuano'], dtype=object)

# NAIVE BAYES model using pipeline

### A machine learning model that is used for large volumes of data, even if you are working with data that has millions of data records the recommended approach is Naive Bayes

In [ ]:
model_pipe5 = pipeline.Pipeline([('vec', vec), ('clf',  naive_bayes.MultinomialNB())])

In [ ]:
model_pipe5.fit(X_train, Y_train)

Pipeline(steps=[('vec', TfidfVectorizer(analyzer='char', ngram_range=(2, 3))),
                ('clf', MultinomialNB())])

In [ ]:
predict_val5 = model_pipe5.predict(X_test)

In [ ]:
metrics.accuracy_score(Y_test,predict_val5)*100

92.55

In [ ]:
metrics.confusion_matrix(Y_test, predict_val5)

array([[1788,  191],
       [ 107, 1914]])

In [ ]:
model_pipe5.predict(['AMbot talga bhie, bakit mna siya ingun ana'])

array(['Cebuano'], dtype='<U8')

# SEPERATED ACCURACY SCORES FROM CELLS

In [ ]:
LR = metrics.accuracy_score(Y_test,predict_val)*100
SVMz = metrics.accuracy_score(Y_test,predict_val2)*100
KNNz = metrics.accuracy_score(Y_test,predict_val3)*100
DTz = metrics.accuracy_score(Y_test,predict_val4)*100
NBz = metrics.accuracy_score(Y_test,predict_val5)*100

In [ ]:
print("Logistic Regression Accuracy Score -> ", LR, "\n")
print("SVM Accuracy Score -> ", SVMz, "\n")
print("KNN Accuracy Score -> ", KNNz, "\n")
print("Decision Tree Accuracy Score -> ", DTz, "\n")
print("Naive Bayes Accuracy Score -> ", NBz)

Logistic Regression Accuracy Score ->  94.825 

SVM Accuracy Score ->  95.39999999999999 

KNN Accuracy Score ->  84.025 

Decision Tree Accuracy Score ->  86.325 

Naive Bayes Accuracy Score ->  92.55


------------------------------------------------------------------------------------------

------------------------------------------------------------------------------------------

# EXPERIMENTS SEC

# VECTORIZING 

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df['Contexts'])
Train_X_Tfidf = Tfidf_vect.transform(X_train)
Test_X_Tfidf = Tfidf_vect.transform(X_test)

# SVM model

In [ ]:
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Y_train)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Y_test)*100)

SVM Accuracy Score ->  96.3


# KNN model

In [ ]:
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(Train_X_Tfidf,Y_train)
# predict the labels on validation dataset
predictions_KNN = neigh.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print("KNN Accuracy Score -> ",accuracy_score(predictions_KNN, Y_test)*100)

KNN Accuracy Score ->  66.975


# DECISION TREE model

In [ ]:
DT = DecisionTreeClassifier(random_state=0)
DT.fit(Train_X_Tfidf,Y_train)
# predict the labels on validation dataset
predictions_DT = DT.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print("DT Accuracy Score -> ",accuracy_score(predictions_DT, Y_test)*100)

DT Accuracy Score ->  89.825


# NAIVE BAYES model

In [ ]:
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Y_train)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X_Tfidf)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ", accuracy_score(predictions_NB, Y_test)*100)

Naive Bayes Accuracy Score ->  96.975


# TEST CASES

In [ ]:
#Test Case
input = ["AMbot talga bhie, bakit mna siya ingun ana"]
transformed_input = Tfidf_vect.transform(input)

In [ ]:
SVM.predict(transformed_input)

array(['Cebuano'], dtype=object)

In [ ]:
neigh.predict(transformed_input)

array(['Filipino'], dtype=object)

In [ ]:
DT.predict(transformed_input)

array(['Filipino'], dtype=object)

In [ ]:
Naive.predict(transformed_input)

array(['Cebuano'], dtype='<U8')

# IDEA

The idea is to train and test the data given, then identify an input in which the input is closest to any data being trained. 